<a href="https://colab.research.google.com/github/adidonner/Machine-Deep-Leap-learning-Python-Based/blob/main/Task7_Cat_vs_Dogs_VGG_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Take the code in keras and write it in pytorch:

In [ ]:
pip install torch torchvision


In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import vgg16

# Set random seed for reproducibility
torch.manual_seed(5)

# Data directory
directory = '/content/drive/MyDrive/cats_vs_dogs_data'

# Data transforms
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
])

# Load datasets
train_dataset = ImageFolder(root=directory, transform=transform)
val_dataset = ImageFolder(root=directory, transform=transform)

# Split datasets into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load pre-trained VGG16 model without top layers
base_model = vgg16(pretrained=True)
base_model.features = base_model.features[:24]  # Equivalent to Keras base_model.layers[:3]

# Freeze the first 3 layers except for fully connected layers and the last one
for param in base_model.parameters():
    param.requires_grad = False
for param in base_model.classifier.parameters():
    param.requires_grad = True

# Modify the classifier to match Keras model
base_model.classifier = nn.Sequential(
    nn.Flatten(),
    nn.Linear(512 * 7 * 7, 8),  # Fixed number of input features
    nn.ReLU(),
    nn.Linear(8, 3),
    nn.Softmax(dim=1)
)

# Create the complete model
model = base_model

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100*correct/total:.2f}%')

# Save the model weights
torch.save(model.state_dict(), '/content/drive/MyDrive/vgg16_frozen_weights_pytorch.pth')


Epoch 1/10, Loss: 0.5827, Accuracy: 96.60%
Epoch 2/10, Loss: 0.5827, Accuracy: 96.60%
Epoch 3/10, Loss: 0.5827, Accuracy: 96.60%
Epoch 4/10, Loss: 0.5827, Accuracy: 96.60%
Epoch 5/10, Loss: 0.5827, Accuracy: 96.60%
Epoch 6/10, Loss: 0.5827, Accuracy: 96.60%
Epoch 7/10, Loss: 0.5827, Accuracy: 96.60%
Epoch 8/10, Loss: 0.5827, Accuracy: 96.60%
Epoch 9/10, Loss: 0.5827, Accuracy: 96.60%
Epoch 10/10, Loss: 0.5827, Accuracy: 96.60%
